#### generate m6A abundance matrix

In [1]:
import os
import pandas as pd

In [2]:
data_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/HeLa_cell/combined_results/results/MatrixEQTL_result"
m6a_matrix = os.path.join(data_dir, "bedtools_quantification.matrix")
os.chdir(data_dir)

In [3]:
df = pd.read_table(m6a_matrix, sep="\t")
bed_file = "hg38_m6Amatrix.bed"
with open(bed_file, 'w') as fw:
    for x in df.index.tolist():
        # in order to matching
        bed_line = "%s\t%s\t%s\t%s" % (x.split(":")[0], x.split(":")[1].split("-")[0], x.split("-")[1].strip(), x.strip())
        fw.write(bed_line + "\n")

In [4]:
%%bash
hg38_bed=hg38_m6Amatrix.bed
hg19_bed=hg19_m6Amatrix.bed
unmapped_bed=unmapped.bed
chain_file=/data/database/chain_file/hg38ToHg19.over.chain
liftOver $hg38_bed $chain_file $hg19_bed $unmapped_bed

Reading liftover chains
Mapping coordinates


In [5]:
df.head()

,HeLa_01,HeLa_02,HeLa_03,HeLa_04,HeLa_05,HeLa_06,HeLa_07,HeLa_08,HeLa_09,HeLa_10,HeLa_11,HeLa_12,HeLa_13,HeLa_14,HeLa_15,HeLa_16
chr8:40153414-40154625,0.538113,1.006682,8.096520,9.064683,10.653016,4.963064,11.631622,17.317888,4.796017,4.169224,3.845882,8.709373,9.085666,8.020432,3.058206,5.495821
chr5:172334066-172335316,0.664089,1.015084,6.207203,6.631909,6.102155,2.878968,1.571427,2.088330,2.838839,2.195517,1.209731,2.281207,1.631635,1.256419,1.739456,2.933057
chr16:46928815-46930453,0.671419,1.189949,2.831050,3.148784,3.112060,1.797235,0.510891,0.776471,2.238562,1.671577,1.309307,2.258178,1.592884,1.244928,1.363850,2.182791
chr6:89643511-89644216,0.684190,0.965920,4.726068,4.444011,3.213446,1.723473,1.641062,2.910035,1.980402,1.598916,0.715831,1.222273,0.749198,0.676402,1.195761,2.012460
chr9:27455132-27455795,0.867411,1.151983,3.619904,3.744498,4.159758,2.155304,1.447960,1.739681,3.378559,2.513339,2.848819,6.168314,4.935373,4.327435,2.211527,3.821381


In [6]:
df.head()
new_title = ["SRR6714636-", "SRR6714637-", "SRR847370-", "SRR847371-", "SRR847358-", "SRR847359-", \
"SRR6714644-", "SRR6714645-", "SRR6714640-", "SRR6714641-", "SRR5925299-", "SRR5925300-", \
"SRR5925287-", "SRR5925288-", "SRR5925271-", "SRR5925272-"]
assembl_dict = {}
with open("hg19_m6Amatrix.bed", 'r') as f:
    for line in f.readlines():
        hg38 = line.strip().split("\t")[-1]
        assembl_dict[hg38] = "%s:%s-%s" % (line.split("\t")[0], line.split("\t")[1], line.split("\t")[2])
##
df.columns = new_title
df_new = df[["SRR5925271-","SRR5925272-","SRR5925287-","SRR5925288-","SRR5925299-","SRR5925300-",\
             "SRR6714636-","SRR6714637-","SRR6714640-","SRR6714641-", "SRR6714644-","SRR6714645-", \
             "SRR847358-","SRR847359-","SRR847370-","SRR847371-"]]
##
with open("hg19_quantification.matrix", 'w') as fw:
    fw.write("id\t" + "\t".join(df_new.columns) + "\n")
    for index, values in df_new.iterrows():
        if index in assembl_dict:
            fw.write("%s\t%s\n" % (assembl_dict[index], "\t".join([str(x) for x in values])))

In [7]:
with open("hg19_m6aloc.txt", 'w') as fw:
    with open("hg19_quantification.matrix", 'r') as f:
        f.readline()
        fw.write("m6aid\tchr\tleft\tright\n")
        for line in f.readlines():
            m6aid = line.split("\t")[0]
            chr_name, start, end = m6aid.split(":")[0],m6aid.split(":")[1].split("-")[0],m6aid.split("-")[1].strip()
            fw.write("\t".join([m6aid, chr_name, start, end]) + "\n")

##### reformat file

In [8]:
with open("hg19_SNP.matrix", 'w') as fw:
    with open("final_SNP.txt", 'r') as f:
        title = f.readline()
        fw.write(title.strip() + "\n")
        for line in f.readlines():
            fw.write("chr" + line)

In [12]:
with open("hg19_SNP.csv", 'w') as fw:
    with open("hg19_SNP.matrix", 'r') as f:
        for line in f.readlines():
            fw.write(",".join(line.split()) + "\n")

In [9]:
with open("hg19_snpsloc.txt", 'w') as fw:
    with open("filtered_snpsloc.txt", 'r') as f:
        title = f.readline()
        fw.write(title.strip() + "\n")
        for line in f.readlines():
            a,b,c = line.strip().split("\t")
            fw.write("chr%s\tchr%s\t%s\n" % (a, b, c))

#### smartPCA

In [14]:
in_file = "hg19_quantification.matrix"
df_m = pd.read_table(in_file, sep="\t")
print(df_m.head())
df_n = df_m.T
df_n.to_csv("hg19_quantification_T.matrix", index=False, header=False)

                         id  SRR5925271-  SRR5925272-  SRR5925287-  \
0    chr8:40010933-40012144     3.058206     5.495821     9.085666   
1  chr5:171761070-171762320     1.739456     2.933057     1.631635   
2   chr16:46962727-46964365     1.363850     2.182791     1.592884   
3    chr6:90353230-90353935     1.195761     2.012460     0.749198   
4    chr9:27455130-27455793     2.211527     3.821381     4.935373   

   SRR5925288-  SRR5925299-  SRR5925300-  SRR6714636-  SRR6714637-  \
0     8.020432     3.845882     8.709373     0.538113     1.006682   
1     1.256419     1.209731     2.281207     0.664089     1.015084   
2     1.244928     1.309307     2.258178     0.671419     1.189949   
3     0.676402     0.715831     1.222273     0.684190     0.965920   
4     4.327435     2.848819     6.168314     0.867411     1.151983   

   SRR6714640-  SRR6714641-  SRR6714644-  SRR6714645-  SRR847358-  SRR847359-  \
0     4.796017     4.169224    11.631622    17.317888   10.653016    4.963064

In [33]:
map_file = "../combined.map"
with open("../combined_2.map", 'w') as fw:
    with open(map_file, 'r') as f:
        for line in f.readlines():
            tmp_list = line.split()
            tmp_list[1] = str(tmp_list[0]) + ":" + str(tmp_list[3])
            fw.write("\t".join(tmp_list) + "\n")